# Baseline

В качестве бейзлайна используется модель NER, состоящая из RNN поверх эмбеддингов FastText (для получения эмбеддингов нужно запустить ноутбук `train_fasttext.ipynb`)

Нормализация брендов и товаров не производится

Бейзлайн реализован на библиотеке PyTorch с использованием PyTorch-Lightning для упрощения кода

In [1]:
from gensim.models.fasttext import FastText
import pandas as pd
import pytorch_lightning as pl
from seqeval.metrics.sequence_labeling import get_entities
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from torchcrf import CRF

torch.set_float32_matmul_precision("high")

2023-06-18 08:44:54.127459: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-18 08:44:54.180160: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-18 08:44:55.049518: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from preprocc import preprocess_text

In [3]:
# !pip install seqeval==1.2.2
# !pip install allennlp==2.10.1
# !pip install pytorch-crf

# Utils

Полезные функции для работы с BIO-тегами

In [4]:
def apply_bio_tagging(row):
    """
    По токенам чека и разметке (то есть выделенным товарам и брендам) строим BIO-теги
    """
    tokens = row["tokens"]
    good = row["good"].split(',')[0].split()
    brand = row["brand"].split(',')[0].split()
    tags = ['O'] * len(tokens)
    for i, token in enumerate(tokens):
        if len(good) > 0 and tokens[i:i + len(good)] == good:
            tags[i] = "B-GOOD"
            for j in range(i + 1, i + len(good)):
                tags[j] = "I-GOOD"
        if len(brand) > 0 and tokens[i:i + len(brand)] == brand:
            tags[i] = "B-BRAND"
            for j in range(i + 1, i + len(brand)):
                tags[j] = "I-BRAND"
    return tags

Прямое и обратное преобразование тегов в индексы

In [5]:
index_to_tag = ["O", "B-GOOD", "I-GOOD", "B-BRAND", "I-BRAND", "PAD"]
tag_to_index = {tag: index for index, tag in enumerate(index_to_tag)}

In [25]:
import re

from cyrtranslit import to_cyrillic, to_latin
lat_to_cyr = str.maketrans("aekmhopctyx", "аекмнорстух")
cyr_to_lat = str.maketrans("аекмнорстух", "aekmhopctyx")
stop_words = set(['г', 'кг', 'шт', 'мл', 'л', 'литр', 'мг', 'гр', 'км', 'мм', 'mm', 'уп'])

def replace_camel_case(s):
    matches = len(re.findall(r'(?<=[a-zа-я])([A-ZА-Я])', s))
    if matches > 2:
        return re.sub(r'(?<=[a-zа-я])([A-ZА-Я])', r' \1', s)
    else:
        return s
    
def split_on_language_change(s):
    s = re.sub(r'(?<=[a-zа-я])(?=[A-ZА-Я0-9])', r' ', s)
    s = re.sub(r'(?<=[A-ZА-Я0-9])(?=[a-zа-я])', r' ', s)
    return s


def insert_space_after_one(s):
    return re.sub(r'(1)(?=[A-Za-zА-Яа-я])', r'\1 ', s)

def replace_zero(s):
    s = re.sub(r'(?<=[A-Za-z])0(?=[A-Za-z])', 'o', s)
    s = re.sub(r'(?<=[А-Яа-я])0(?=[А-Яа-я])', 'о', s)
    return s

def preprocess_text(text):
    
    text = replace_zero(text)
    
    text = re.sub('\d+', '1', text)  # replace numbers to 1
    text = replace_camel_case(text)
    text = re.sub('д/', 'для ', text)
    text = re.sub('Д/', 'для ', text)
    text = insert_space_after_one(text)
    text = re.sub(r'\s+', ' ', text)  # remove extra spaces
    text = re.sub(r'[^\w\s]', ' ', text)  # remove punctuation
    words = []
    for w in text.lower().split():
#         if len(w) < 2:
#             continue
        
        num_eng_chars = len(re.findall(r'[a-z]', w))
        num_ru_chars = len(re.findall(r'[а-я]', w))
        if num_eng_chars and num_ru_chars:
            if num_eng_chars > num_ru_chars:
                w = w.translate(cyr_to_lat)
            else:
                w = w.translate(lat_to_cyr)
        
        if w in stop_words:
            continue
        
        # если нет транзиторов
        w = split_on_language_change(w)
        if ' ' in w:
            words.extend(w.split())
        else:
            if w == 'нести':
                w = 'nestea'
            if w == 'эпика':
                w = 'epica'
            if w == 'хелен':
                w = 'helen'
            if w == 'харпер':
                w = 'harper'
            if w == 'тимотей':
                w = 'timotei'
            if w == 'тесс':
                w = 'tess'
            if w == 'кроненбург':
                w = 'kronenbourg'
            if w == 'пай':
                w = 'pie'
            if w == 'чоко':
                w = 'choko'
            if w == 'салтон':
                w = 'salton'
            words.append(w)
        
    return words

# Datamodule

Подготовим данные для модели. Для этого определим наследника `torch.nn.utils.Dataset` - `ReceiptsDataset`

In [26]:
class ReceiptsDataset(Dataset):
    def __init__(self, df, fasttext):
        super().__init__()
        self.is_predict = "tags" not in df.columns
        self.data = df[["tokens", "good", "brand", "tags"]] if not self.is_predict else df[["tokens", "id"]]
        self.data = self.data.values
        self.fasttext = fasttext

    def __getitem__(self, index):
        identifier = 0 if not self.is_predict else self.data[index][1]
        tokens = self.data[index][0]
        embeddings = self.fasttext.wv[tokens]
        goods = self.data[index][1].split(',') if not self.is_predict else list()
        brands = self.data[index][2].split(',') if not self.is_predict else list()
        tags = self.data[index][3] if not self.is_predict else ["O"] * len(tokens)
        target = [tag_to_index[tag] for tag in tags]
        return identifier, tokens, embeddings, goods, brands, target

    def __len__(self):
        return len(self.data)

Для объединения примеров в батчи нужна специальная `collate_fn`, в которой происходит паддинг

In [27]:
def collate_fn(batch):
    ids, tokens_sequence, embeddings_sequence, goods, brands, targets = list(zip(*batch))
    embeddings_sequence = pad_sequence([torch.FloatTensor(sequence) for sequence in embeddings_sequence],
                                       batch_first=True)
    targets = pad_sequence([torch.LongTensor(target) for target in targets], batch_first=True,
                           padding_value=tag_to_index["PAD"])
    return ids, tokens_sequence, embeddings_sequence, goods, brands, targets

Используем LightningDataModule для задания пайплайна

1. prepare_data
    1. Токенизируем текст
    2. Выделяем BIO-теги в размеченной части
2. setup
    1. Разделяем размеченную выборку на обучающую и валидационную
    2. Создаем `ReceiptsDataset` под каждую выборку

In [28]:
class ReceiptsDataModule(pl.LightningDataModule):
    def __init__(self,
                 train_dataset_path,
                 test_dataset_path,
                 fasttext_path,
                 val_split_size,
                 batch_size,
                 num_workers):
        super().__init__()
        self.train_dataset_path = train_dataset_path
        self.test_dataset_path = test_dataset_path
        self.fasttext_path = fasttext_path
        self.val_split_size = val_split_size
        self.batch_size = batch_size
        self.num_workers = num_workers

    def prepare_data(self):
        self.fasttext = FastText.load(self.fasttext_path)
        self.train_df = pd.read_csv(self.train_dataset_path).fillna("")
        self.test_df = pd.read_csv(self.test_dataset_path)
        self.train_df["tokens"] = self.train_df["name"].apply(preprocess_text)
        self.test_df["tokens"] = self.test_df["name"].apply(preprocess_text)

        self.train_df["tags"] = self.train_df.apply(apply_bio_tagging, axis=1)

    def setup(self, stage: str):
        self.train_df, self.val_df = train_test_split(self.train_df, test_size=self.val_split_size, random_state=42)

        self.train_dataset = ReceiptsDataset(self.train_df, self.fasttext)
        self.val_dataset = ReceiptsDataset(self.val_df, self.fasttext)
        self.predict_dataset = ReceiptsDataset(self.test_df, self.fasttext)

    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers,
                          collate_fn=collate_fn)

    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers,
                          collate_fn=collate_fn)

    def predict_dataloader(self):
        return torch.utils.data.DataLoader(self.predict_dataset,
                                           batch_size=self.batch_size,
                                           num_workers=self.num_workers,
                                           collate_fn=collate_fn)

In [29]:
TRAIN_DATASET_PATH = "data/train_supervised_dataset.csv"
TEST_DATASET_PATH = "data/test_dataset.csv"
FASTTEXT_PATH = "fasttext_models/fasttext_256_hardpreprocc_fitted.model"
FASTTEXT_DIM = 256
VAL_SPLIT_SIZE = 0.1
BATCH_SIZE = 512
NUM_WORKERS = 4

In [30]:
dm = ReceiptsDataModule(
    TRAIN_DATASET_PATH,
    TEST_DATASET_PATH,
    FASTTEXT_PATH,
    VAL_SPLIT_SIZE,
    BATCH_SIZE,
    NUM_WORKERS
)

# Model

Сначала определим метрику `F1` для задачи NER

In [11]:
class F1Score:
    def __init__(self):
        self.tp = 0
        self.fp = 0
        self.fn = 0

    def update(self, pred, target):
        pred = frozenset(x for x in pred)
        target = frozenset(x for x in target)
        self.tp += len(pred & target)
        self.fp += len(pred - target)
        self.fn += len(target - pred)

    def reset(self):
        self.tp = 0
        self.fp = 0
        self.fn = 0

    def get(self):
        if self.tp == 0:
            return 0.0
        precision = self.tp / (self.tp + self.fp)
        recall = self.tp / (self.tp + self.fn)
        return 2 / (1 / precision + 1 / recall)

Зададим саму модель, ее шаги на обучении, валидации и инференсе, а также способ обучения

In [12]:
class ReceiptsModule(pl.LightningModule):
    def __init__(self,
                 rnn_input_size,
                 rnn_hidden_size,
                 rnn_num_layers,
                 rnn_dropout,
                 mlp_hidden_size,
                 learning_rate,
                 rnn_bidir=True,
                 num_tags=-1,
                ):
        super().__init__()
        self.learning_rate = learning_rate
        self.lstm = nn.LSTM(input_size=rnn_input_size,
                           hidden_size=rnn_hidden_size,
                           num_layers=rnn_num_layers,
                           batch_first=True,
                           dropout=rnn_dropout,
                           bidirectional=rnn_bidir,
                           )
        self.mlp = nn.Sequential(
            nn.Linear(rnn_hidden_size + rnn_hidden_size * int(rnn_bidir), mlp_hidden_size),
            nn.GELU(),
            nn.Dropout(p=0.3),
            nn.Linear(mlp_hidden_size, len(index_to_tag)),
        )
        self.crf = CRF(num_tags, batch_first=True)
        self.f1_good_train = F1Score()
        self.f1_brand_train = F1Score()
        self.f1_good_val = F1Score()
        self.f1_brand_val = F1Score()

    def forward(self, sequences):
        sequences, _ = self.lstm(sequences)
        logits = self.mlp(sequences)
        return logits

    def training_step(self, batch, _):
        ids, tokens_sequence, embeddings_sequence, goods, brands, targets = batch
        logits = self(embeddings_sequence)
        loss = -self.crf(logits, targets)
        tags_indices_sequence = self.crf.decode(logits)

        for i, tags_indices in enumerate(tags_indices_sequence):
            tags = [index_to_tag[index] for index in tags_indices[:len(tokens_sequence[i])]]
            entities = get_entities(tags)
            goods_pred = [' '.join(tokens_sequence[i][start:finish + 1]) for t, start, finish in entities if t == "GOOD"]
            brands_pred = [' '.join(tokens_sequence[i][start:finish + 1]) for t, start, finish in entities if t == "BRAND"]
            self.f1_good_train.update(goods_pred, goods[i])
            self.f1_brand_train.update(brands_pred, brands[i])
        self.log("loss/train", loss, on_epoch=True, batch_size=len(tags_indices_sequence))
        return loss

    def on_train_epoch_end(self):
        self.log("metric/f1_good_train", self.f1_good_train.get())
        self.log("metric/f1_brand_train", self.f1_brand_train.get())
        self.f1_good_train.reset()
        self.f1_brand_train.reset()

    def validation_step(self, batch, _):
        ids, tokens_sequence, embeddings_sequence, goods, brands, targets = batch
        logits = self(embeddings_sequence)
        loss = -self.crf(logits, targets)
        tags_indices_sequence = self.crf.decode(logits)

        for i, tags_indices in enumerate(tags_indices_sequence):
            tags = [index_to_tag[index] for index in tags_indices[:len(tokens_sequence[i])]]
            entities = get_entities(tags)
            goods_pred = [' '.join(tokens_sequence[i][start:finish + 1]) for t, start, finish in entities if t == "GOOD"]
            brands_pred = [' '.join(tokens_sequence[i][start:finish + 1]) for t, start, finish in entities if t == "BRAND"]
            self.f1_good_val.update(goods_pred, goods[i])
            self.f1_brand_val.update(brands_pred, brands[i])
        self.log("loss/val", loss, batch_size=len(tags_indices_sequence))

    def on_validation_epoch_end(self):
        self.log("metric/f1_good_val", self.f1_good_val.get())
        self.log("metric/f1_brand_val", self.f1_brand_val.get())
        self.f1_good_val.reset()
        self.f1_brand_val.reset()

    def predict_step(self, batch, _):
        ids, tokens_sequence, embeddings_sequence, _, _, _ = batch
        logits = self(embeddings_sequence)
        tags_indices_sequence = self.crf.decode(logits)
        result = list()
        for i, tags_indices in enumerate(tags_indices_sequence):
            tags = [index_to_tag[index] for index in tags_indices[:len(tokens_sequence[i])]]
            entities = get_entities(tags)
            goods_pred = ','.join([' '.join(tokens_sequence[i][start:finish + 1]) for t, start, finish in entities if t == "GOOD"])
            brands_pred = ','.join([' '.join(tokens_sequence[i][start:finish + 1]) for t, start, finish in entities if t == "BRAND"])
            result.append([ids[i], goods_pred, brands_pred])
        return result

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), self.learning_rate)

In [13]:
FASTTEXT_DIM

256

In [14]:
RNN_INPUT_SIZE = FASTTEXT_DIM
RNN_HIDDEN_SIZE = FASTTEXT_DIM
RNN_NUM_LAYERS = 3
RNN_DROPOUT = 0.3
MLP_HIDDEN_SIZE = 100
LEARNING_RATE = 1e-4
model = ReceiptsModule(
    RNN_INPUT_SIZE,
    RNN_HIDDEN_SIZE,
    RNN_NUM_LAYERS,
    RNN_DROPOUT,
    MLP_HIDDEN_SIZE,
    LEARNING_RATE,
    num_tags=len(tag_to_index),
)

In [16]:
trainer = pl.Trainer(
    accelerator="gpu",
    devices=[0],
    logger=pl.loggers.TensorBoardLogger("tb_logs", name="ner_crf_lstm_3_256_pre_30e"),
    max_epochs=100,
    log_every_n_steps=1
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Обучение модели

In [17]:
trainer.fit(model, datamodule=dm)

Missing logger folder: tb_logs/ner_crf_lstm_3_256_pre_30e
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name | Type       | Params
------------------------------------
0 | lstm | LSTM       | 4.2 M 
1 | mlp  | Sequential | 51.9 K
2 | crf  | CRF        | 48    
------------------------------------
4.3 M     Trainable params
0         Non-trainable params
4.3 M     Total params
17.034    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/worker/.local/lib/python3.9/site-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at /opt/conda/conda-bld/pytorch_1678402374358/work/aten/src/ATen/native/TensorCompare.cpp:493.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Training: 0it [00:00, ?it/s]

/home/worker/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/worker/anaconda3/envs/py/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [22]:
test_df = dm.test_df.copy()

In [24]:
for i, r in test_df.iterrows():
    print(r)

id                                                      0
name      469-210 ЕРМАК Клей универсальный, 15мл, блистер
tokens              [ермак, клей, универсальный, блистер]
Name: 0, dtype: object
id                                      1
name      Торт СЛАДУШКА Зимняя вишня 700г
tokens    [торт, сладушка, зимняя, вишня]
Name: 1, dtype: object
id                                             2
name        Смеситель "CALORIE" 1023 А06 д/кухни
tokens    [смеситель, calorie, а, 1, для, кухни]
Name: 2, dtype: object
id                     3
name      Лимон 50гр БАР
tokens      [лимон, бар]
Name: 3, dtype: object
id                                            4
name      Коньяк САРАДЖИШВИЛИ 5 лет 0,5л Грузия
tokens      [коньяк, сараджишвили, лет, грузия]
Name: 4, dtype: object
id                                                        5
name      Born Pretty, Пластина для стемпинга BP-L011 Te...
tokens    [born, pretty, пластина, для, стемпинга, bp, l...
Name: 5, dtype: object
id              

id                                                     455
name      2: 891-082. VETTA Коврик силиконовый для раскатк
tokens          [vetta, коврик, силиконовый, для, раскатк]
Name: 455, dtype: object
id                                          456
name      Святой  источник  н/газ  спорт  0,75л
tokens           [святой, источник, газ, спорт]
Name: 456, dtype: object
id                                         457
name      3 2456546789431 Н/К Куртка/Mass/3417
tokens                          [куртка, mass]
Name: 457, dtype: object
id                                        458
name      Фильтр воздушный AUDI Q7 (4M) 2015-
tokens        [фильтр, воздушный, audi, q, 1]
Name: 458, dtype: object
id                        459
name         Гребешки крупные
tokens    [гребешки, крупные]
Name: 459, dtype: object
id                                                      460
name      Сайлентблок рычага 2101-07 верхний БРТ шт код ...
tokens             [сайлентблок, рычага, верхний, брт, код]
Name:

id                                  940
name      1 КГ СОУС HEINZ 1000 ОСТРОВОВ
tokens          [соус, heinz, островов]
Name: 940, dtype: object
id                                                      941
name      АКЦИЯ Коробка на 6 капкейков с окном (белая), ...
tokens        [акция, коробка, на, капкейков, окном, белая]
Name: 941, dtype: object
id                          942
name      Креветка тигровая 1кг
tokens     [креветка, тигровая]
Name: 942, dtype: object
id                                   943
name      Газ/напиток Ниагара Дюшес 1,5л
tokens    [газ, напиток, ниагара, дюшес]
Name: 943, dtype: object
id                                           944
name      Говядина СохранимТрадиции 525г тушеная
tokens     [говядина, сохранимтрадиции, тушеная]
Name: 944, dtype: object
id                                945
name      Футболка HAYAS кр.р. PINK-L
tokens    [футболка, hayas, кр, pink]
Name: 945, dtype: object
id                                        946
name        Фреза пламя 

id                                                     1450
name      Батончик Леовит Худеем за неделю Яблоко/корица...
tokens    [батончик, леовит, худеем, за, неделю, яблоко,...
Name: 1450, dtype: object
id                                                 1451
name      Футболка Короткий рукав CL 1053338 черный(XS)
tokens      [футболка, короткий, рукав, cl, черный, xs]
Name: 1451, dtype: object
id                                                     1452
name      Держатель для счетов алюминиевый 450 мм [BH-18...
tokens            [держатель, для, счетов, алюминиевый, bh]
Name: 1452, dtype: object
id                             1453
name      Комплект Ferz LM087408947
tokens      [комплект, ferz, lm, 1]
Name: 1453, dtype: object
id                                            1454
name      МАЙОНЕЗ "МАХЕЕВ" С ПЕРЕПЕЛИНЫМ ЯЙЦОМ 400
tokens       [майонез, махеев, перепелиным, яйцом]
Name: 1454, dtype: object
id                                                     1455
name      Повязка на/

id                                                     1991
name      Пастернак Круглый, семена Агроуспех 1г (Хороши...
tokens    [пастернак, круглый, семена, агроуспех, хороши...
Name: 1991, dtype: object
id                                                     1992
name      Пробка топливного бака, бензобака 8450006355 L...
tokens    [пробка, топливного, бака, бензобака, lada, la...
Name: 1992, dtype: object
id                                        1993
name      7 2157325814497 ФУТБОЛКА Ж/DESI/52T2
tokens                  [футболка, desi, t, 1]
Name: 1993, dtype: object
id                                       1994
name      ДИСК АЛМАЗНЫЙ VERTEX 125мм  СЕГМЕНТ
tokens      [диск, алмазный, vertex, сегмент]
Name: 1994, dtype: object
id                                             1995
name        Овощечистка поворотная 2лезвия mfk01496
tokens    [овощечистка, поворотная, лезвия, mfk, 1]
Name: 1995, dtype: object
id                                        1996
name      Горчица Русская 17

id                                    2550
name        Контейнер д/салата 0,5л низкий
tokens    [контейнер, для, салата, низкий]
Name: 2550, dtype: object
id                    2551
name        Водолазка муж.
tokens    [водолазка, муж]
Name: 2551, dtype: object
id                                                     2552
name      Батончик фруктовый 20г яблоко-курага-фенхель-з...
tokens    [батончик, фруктовый, яблоко, курага, фенхель,...
Name: 2552, dtype: object
id                                           2553
name      Попкорн "ДВОЙНАЯ КАРАМЕЛЬ" 100гр. стак.
tokens         [попкорн, двойная, карамель, стак]
Name: 2553, dtype: object
id                                                2554
name      Круасcаны 7 days вареная сгущенка, 4шт х 65г
tokens            [круассаны, days, вареная, сгущенка]
Name: 2554, dtype: object
id                                              2555
name         Средство д/мытья посуды АОС лимон 450мл
tokens    [средство, для, мытья, посуды, аос, лимон]
Name: 

id                              3104
name      6526 Майонез МЖК 67% 600мл
tokens                [майонез, мжк]
Name: 3104, dtype: object
id                                                     3105
name      4.ТРИКОТАЖНАЯ ФУТБОЛКА - БОДИ С ДЛ. 0WH601Z8 8...
tokens       [трикотажная, футболка, боди, дл, wh, 1, z, 1]
Name: 3105, dtype: object
id                                    3106
name      Зубная паста НЖ фтор  136гр (32)
tokens           [зубная, паста, нж, фтор]
Name: 3106, dtype: object
id                         3107
name      Икра кеты 0,5 элитная
tokens    [икра, кеты, элитная]
Name: 3107, dtype: object
id                                        3108
name      3 2456551430106 Пальто жен/SHAR/SRR9
tokens             [пальто, жен, shar, srr, 1]
Name: 3108, dtype: object
id                                                    3109
name             Пластырь фиксир SFM  2смX500см тканев осн
tokens    [пластырь, фиксир, sfm, смх, 1, см, тканев, осн]
Name: 3109, dtype: object
id        

id                               3658
name      ЯНТАРНАЯ СОЛОМКА 25Г BEERKA
tokens    [янтарная, соломка, beerka]
Name: 3658, dtype: object
id                                                     3659
name      JO Flavored H2O Ананас гель на водной основе, ...
tokens    [jo, flavored, h, 1, ананас, гель, на, водной,...
Name: 3659, dtype: object
id                                                     3660
name      ВОДА ГАЗ КАМЧАТСКИЕ ВОДЫ КАМЧАТКА МИНЕРАЛЬНАЯ ...
tokens    [вода, газ, камчатские, воды, камчатка, минера...
Name: 3660, dtype: object
id                                  3661
name      Спатифиллум Шопен 9/30 оз 1х18
tokens    [спатифиллум, шопен, оз, х, 1]
Name: 3661, dtype: object
id                                3662
name      Зефир Ванильный 280гр Х/з №4
tokens              [зефир, ванильный]
Name: 3662, dtype: object
id                   3663
name      Ассам чай 350мл
tokens       [ассам, чай]
Name: 3663, dtype: object
id                                             3664


id                            4198
name      Земляника (лист, 35 гр).
tokens           [земляника, лист]
Name: 4198, dtype: object
id                                     4199
name       Шайба М14 усиленная DIN 9021 оц.
tokens    [шайба, м, 1, усиленная, din, оц]
Name: 4199, dtype: object
id                                                     4200
name      8699546493104:Средство от насекомых (шт), Прес...
tokens               [средство, от, насекомых, престиж, кс]
Name: 4200, dtype: object
id                                 4201
name      SL Лимонад Классический 1/250
tokens      [sl, лимонад, классический]
Name: 4201, dtype: object
id                                                4202
name      Стакан бум С ДР Мозаика 250мл 6шт/Сфера упак
tokens         [стакан, бум, др, мозаика, сфера, упак]
Name: 4202, dtype: object
id                                                     4203
name      401892    Отвертка аккумуляторная Вихрь ОА-3,6...
tokens    [отвертка, аккумуляторная, вихрь, оа, 

id                         4742
name      Талка  водка 0,7л.40%
tokens           [талка, водка]
Name: 4742, dtype: object
id                             4743
name       Заглушка 60х60 (пластик)
tokens    [заглушка, х, 1, пластик]
Name: 4743, dtype: object
id                                    4744
name      132403 ПУЛЬТ ДУ TOSHIBA CT-90298
tokens            [пульт, ду, toshiba, ct]
Name: 4744, dtype: object
id                                                     4745
name      Цианокобаламин (В 12) 0,2/ 0,5 мг/мл амп.1 мл ...
tokens                                [цианокобаламин, амп]
Name: 4745, dtype: object
id                                            4746
name      588204ЧНР Ботинки мужские MRX, 46, Черны
tokens         [чнр, ботинки, мужские, mrx, черны]
Name: 4746, dtype: object
id                                          4747
name      Батон Любимый нарезан 300гр Полюс Торг
tokens    [батон, любимый, нарезан, полюс, торг]
Name: 4747, dtype: object
id                             

Получение итоговых сущностей для тестового датасета

In [31]:
pred = trainer.predict(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Predicting: 27it [00:00, ?it/s]

/home/worker/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [34]:
submission = pd.DataFrame(sum(pred, list()), columns=["id", "good", "brand"])
submission

,id,good,brand
0,0,клей,ермак
1,1,торт,
2,2,смеситель,calorie
3,3,,
4,4,коньяк,
...,...,...,...
4995,4995,рамка,
4996,4996,напиток,
4997,4997,наконечники,
4998,4998,шоколад,риттерспорт


In [23]:
test_df = dm.test_df.copy()

In [26]:
test_df['pred_good'] = submission['good']

In [27]:
test_df['pred_brand'] = submission['brand']

In [32]:
dm.train_df[['name', 'tokens', 'tags']]

,name,tokens,tags
17030,954159 Салфетки влаж.Aura Tropic coctail антиб...,"[салфетки, влаж, aura, tropic, coctail, антиба...","[B-GOOD, O, B-BRAND, O, O, O, O, O]"
12634,Мороженое Инмарко Золотой стандарт новый пломб...,"[мороженое, инмарко, золотой, стандарт, новый,...","[B-GOOD, O, B-BRAND, I-BRAND, O, O, O, O, O]"
4627,Шампунь д/вол Pantene Аква Лайт Питательный 250мл,"[шампунь, д, вол, pantene, аква, лайт, питател...","[B-GOOD, O, O, B-BRAND, O, O, O, O]"
17500,Пиво Хамовники Мюнхенское 5.5%0.45л.ж/бРШТ,"[пиво, хамовники, мюнхенское, л, ж, бршт]","[B-GOOD, B-BRAND, O, O, O, O]"
23095,Доска гладильная EUROGOLD C2 Johnny Plexus Min...,"[доска, гладильная, eurogold, c, johnny, plexu...","[B-GOOD, O, B-BRAND, O, O, O, O, O, O, O, O, O]"
...,...,...,...
12587,"КЛЮЧ ТРУБН PROFI 90* 2""","[ключ, трубн, profi]","[B-GOOD, O, O]"
17733,5 2456564931386 Колготки ж/OPIU/Stri,"[колготки, ж, opiu, stri]","[B-GOOD, O, O, O]"
1713,3 2456575442604 Толстовка /UCLA/UCHZ,"[толстовка, ucla, uchz]","[B-GOOD, B-BRAND, O]"
9461,Хлеб кирпичик ржаной 26*330г BRIDOR Франция,"[хлеб, кирпичик, ржаной, г, bridor, франция]","[B-GOOD, O, O, O, B-BRAND, O]"


In [35]:
submission.to_csv("submissions/submission_hardpreprocc_renaming_256e30.csv", index=False)